# 寫入資料庫

In [1]:
# 導入庫
import os
import pymysql
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from urllib.parse import quote
from dotenv import load_dotenv

# 載入 .env 檔案中的變數
load_dotenv()

# MariaDB 連線設定
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT", 3306))
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# 自訂函數：插入數據到資料庫
def insert_into_db(data_list):
    connection = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        charset='utf8mb4'
    )

    with connection:
        with connection.cursor() as cursor:
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS coupang_products (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    search_keyword VARCHAR(255),
                    title TEXT,
                    full_text TEXT,
                    price VARCHAR(50),
                    unit_price VARCHAR(50),
                    timestamp DATETIME
                );
            """)
            for data in data_list:
                cursor.execute("""
                    INSERT INTO coupang_products (search_keyword, title, full_text, price, unit_price, timestamp)
                    VALUES (%s, %s, %s, %s, %s, %s);
                """, (
                    data["search_keyword"],
                    data["title"],
                    data["full_text"],
                    data["price"],
                    data["unit_price"],
                    data["timestamp"]
                ))
        connection.commit()

# 自訂函數：數據探勘
def get_coupang_search_results(search_keyword: str, advanced_keywords: list[str]):
    encoded_keyword = quote(search_keyword)
    url = f"https://www.tw.coupang.com/search?q={encoded_keyword}&channel=user"
    # 設定
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--lang=zh-TW")
    # 物件
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    driver.get(url)
    time.sleep(5)
    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    product_cards = soup.select(
        "div.SearchResult_searchResultProduct___h6E9"
    )

    now = datetime.now()
    results = []

    for card in product_cards:
        try:
            full_text = card.get_text(separator=" ", strip=True)
            title_tag = card.select_one("div.Product_title__8K0xk")
            title = title_tag.get_text(strip=True) if title_tag else "N/A"
            price_tag = card.select_one(
                "span.Product_salePricePrice__2FbsL span"
            )
            price = price_tag.get_text(strip=True) if price_tag else "N/A"
            unit_price_tag = card.select_one("div.Product_unitPrice__QQPdR")
            unit_price = unit_price_tag.get_text(strip=True) if unit_price_tag else "N/A"

            product = {
                "search_keyword": search_keyword,
                "title": title,
                "full_text": full_text,
                "price": price,
                "unit_price": unit_price,
                "timestamp": now
            }
            results.append(product)

        except Exception as e:
            print("解析錯誤：", e)
            continue

    return results

if __name__ == "__main__":
    # 關鍵字
    search_keyword = "BLUE BAY 倍力 Sense"
    # 進階關鍵字
    advanced_keywords = []
    # 數據探勘
    results = get_coupang_search_results(
        search_keyword,
        advanced_keywords
    )
    # 寫入資料庫
    insert_into_db(results)
    # 輸出狀態
    print("資料已寫入 MariaDB。")


資料已寫入 MariaDB。


輸出查看

In [2]:
# 輸出結果總數
print(f"\n搜尋關鍵字：{search_keyword}")
print(f"共取得 {len(results)} 筆結果\n")

# 顯示所有結果
for idx, item in enumerate(results, 1):
    print(f"{idx}. 標題: {item['title']}")
    print(f"   價格: {item['price']}")
    print(f"   每單位: {item['unit_price']}")
    print(f"   完整內容: {item['full_text']}")
    print("-" * 60)

# 顯示進階篩選結果（只有當有關鍵字時才執行）
if advanced_keywords:
    print(f"\n進階條件符合項目（包含：{'、'.join(advanced_keywords)}）：")
    filtered = [
        item for item in results
        if all(kw in item["full_text"] for kw in advanced_keywords)
    ]
    for idx, item in enumerate(filtered, 1):
        print(f"{idx}. 標題: {item['title']}")
        print(f"   價格: {item['price']}")
        print(f"   每單位: {item['unit_price']}")
        print(f"   完整內容: {item['full_text']}")
        print("-" * 60)


搜尋關鍵字：味丹 氣泡水
共取得 30 筆結果

1. 標題: 味丹 多喝水 MORE氣泡水 檸檬風味, 560ml, 24瓶
   價格: $287
   每單位: ($2.14/100ml)
   完整內容: 味丹 多喝水 MORE氣泡水 檸檬風味, 560ml, 24瓶 特價 46折 $624 $287 ($2.14/100ml) 7折 優惠券 明天 5/17 (六) 預計送達 免運 滿 $490 ( 919 )
------------------------------------------------------------
2. 標題: 味丹 MORE氣泡水 蜜桃風味, 560ml, 24瓶
   價格: $287
   每單位: ($2.14/100ml)
   完整內容: 味丹 MORE氣泡水 蜜桃風味, 560ml, 24瓶 特價 46折 $624 $287 ($2.14/100ml) 7折 優惠券 明天 5/17 (六) 預計送達 免運 滿 $490 ( 722 )
------------------------------------------------------------
3. 標題: 味丹 究‧選 SUÁN蜜桃紅茶氣泡飲, 540ml, 24瓶
   價格: $384
   每單位: ($2.96/100ml)
   完整內容: 味丹 究‧選 SUÁN蜜桃紅茶氣泡飲, 540ml, 24瓶 特價 56折 $696 $384 ($2.96/100ml) 7折 優惠券 明天 5/17 (六) 預計送達 免運 滿 $490 ( 144 )
------------------------------------------------------------
4. 標題: 味丹 MORE+維他命氣泡水 乳酸風味, 560ml, 24瓶
   價格: $429
   每單位: ($3.19/100ml)
   完整內容: 味丹 MORE+維他命氣泡水 乳酸風味, 560ml, 24瓶 特價 82折 $529 $429 ($3.19/100ml) 7折 優惠券 明天 5/17 (六) 預計送達 免運 滿 $490 ( 5 )
-------------------------------------------------------